##Date:  05/20/2015

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline
import requests
from pattern import web

In [19]:
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk import sent_tokenize
from nltk.corpus import stopwords
import re

In [20]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from unidecode import unidecode
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

##1. Process speakers info 

read in the train.json file

In [21]:
talks_train = pd.read_json('train.json')

In [22]:
talks_train.reset_index(drop = True, inplace = True)

In [23]:
talks_train.shape

(1535, 31)

In [24]:
# select a talk with multi-speakers as test
test = talks_train.ix[23,'speaker_ids']

In [25]:
test

[1057, 685]

### get all speakers' info 

In [4]:
url_speaker = 'https://api.ted.com/v1/speakers.json?api-key=ynw2u8e4h9sk8c2htp7vutxq'

In [5]:
params = dict(limit = 100, offset = 0)
response = requests.get(url_speaker,params = params)

In [6]:
response.json()

{u'counts': {u'this': 100, u'total': 1696},
 u'speakers': [{u'speaker': {u'description': u'Climate-change prophet',
    u'firstname': u'Al',
    u'id': 2,
    u'lastname': u'Gore',
    u'middleinitial': u'',
    u'published_at': u'2007-02-13 10:43:00',
    u'slug': u'al_gore',
    u'title': u'',
    u'updated_at': u'2009-04-08 14:44:57',
    u'whotheyare': u"Once the US Vice President, then star of <em>An Inconvenient Truth</em>, now Nobel Peace Prize winner, Al Gore found a way to focus the world's attention on climate change. In doing so, he has invented a new medium -- the Keynote movie -- and reinvented himself.",
    u'whylisten': u'<p>Unlike some in public office, Al Gore always intended to get something done, and since leaving Washington, DC -- following the tumultuous 2000 election -- he&#39;s still at it. In fact, <strong>his campaign for alerting the world to the dangers of climate change</strong> has only gained momentum. His Oscar-winning <a href="http://www.climatecrisis.n

In [7]:
speaker_collection = []
for i in xrange(17):
    offset = i*100
    response = requests.get(url_speaker,params = dict(limit = 100, offset = offset))
    speaker_collection.append(response.json())

In [8]:
speakers = [x['speaker'] for temp in speaker_collection for x in temp['speakers']]

In [9]:
len(speakers)

1696

In [10]:
speakers[0].keys()

[u'description',
 u'firstname',
 u'title',
 u'updated_at',
 u'lastname',
 u'middleinitial',
 u'published_at',
 u'whylisten',
 u'slug',
 u'whotheyare',
 u'id']

In [11]:
speakers = pd.DataFrame(speakers)

In [12]:
speakers.set_index('id',drop = True,inplace = True)

In [13]:
speakers.whotheyare = speakers.whotheyare.apply(lambda x: re.sub("<.*?>", "", x))

In [14]:
speakers.whylisten = speakers.whylisten.apply(lambda x: re.sub("<.*?>", "", x))

In [15]:
speakers.head()

,description,firstname,lastname,middleinitial,published_at,slug,title,updated_at,whotheyare,whylisten
id,,,,,,,,,,
2,Climate-change prophet,Al,Gore,,2007-02-13 10:43:00,al_gore,,2009-04-08 14:44:57,"Once the US Vice President, then star of An In...","Unlike some in public office, Al Gore always i..."
3,"inventor, engineer",Amy,Smith,,2007-02-13 10:46:00,amy_smith,,2010-06-30 11:15:45,"Amy Smith designs cheap, practical fixes for t...",Mechanical engineer Amy Smith&#39;s approach t...
4,President-elect of Afghanistan,Ashraf,Ghani,,2007-02-13 10:47:00,ashraf_ghani,,2015-03-04 03:50:59,"Ashraf Ghani, Afghanistan’s new president-elec...",Ashraf Ghani became Afghanistan&rsquo;s new pr...
5,Aircraft engineer,Burt,Rutan,,2007-02-13 10:49:00,burt_rutan,,2010-06-15 15:58:14,"In 2004, legendary spacecraft designer Burt Ru...",Burt Rutan is widely regarded as one of the wo...
6,Car designer,Chris,Bangle,,2007-02-13 10:59:00,chris_bangle,,2010-06-15 16:00:03,Car design is a ubiquitous but often overlooke...,American designer Chris Bangle understands tha...


In [16]:
speakers.reset_index(inplace = True)

In [17]:
speakers.to_json('speakers.json')

In [26]:
speakers = pd.read_json('speakers.json')

In [27]:
speakers.sort(columns = 'id',inplace = True)

In [28]:
speakers.set_index('id',inplace = True)

In [29]:
speakers.head()

,description,firstname,lastname,middleinitial,published_at,slug,title,updated_at,whotheyare,whylisten
id,,,,,,,,,,
2,Climate-change prophet,Al,Gore,,2007-02-13 10:43:00,al_gore,,2009-04-08 14:44:57,"Once the US Vice President, then star of An In...","Unlike some in public office, Al Gore always i..."
3,"inventor, engineer",Amy,Smith,,2007-02-13 10:46:00,amy_smith,,2010-06-30 11:15:45,"Amy Smith designs cheap, practical fixes for t...",Mechanical engineer Amy Smith&#39;s approach t...
4,President-elect of Afghanistan,Ashraf,Ghani,,2007-02-13 10:47:00,ashraf_ghani,,2015-03-04 03:50:59,"Ashraf Ghani, Afghanistan’s new president-elec...",Ashraf Ghani became Afghanistan&rsquo;s new pr...
5,Aircraft engineer,Burt,Rutan,,2007-02-13 10:49:00,burt_rutan,,2010-06-15 15:58:14,"In 2004, legendary spacecraft designer Burt Ru...",Burt Rutan is widely regarded as one of the wo...
6,Car designer,Chris,Bangle,,2007-02-13 10:59:00,chris_bangle,,2010-06-15 16:00:03,Car design is a ubiquitous but often overlooke...,American designer Chris Bangle understands tha...


###1.1 Determine gender 

In [30]:
genderdict = {'he':1,'him':1,'his':1,'she':0,'her':0,'hers':0}

In [31]:
def genderdet(text):
    found = False
    for word in text.split(' '):
        word = word.lower()
        if genderdict.has_key(word):
            found = True
            return genderdict[word]
    if not found:
        return -1

In [32]:
# define a function to get speakers info given speakers ids
def speakerinfo(ids):
    # ids is a list of ids
    gender = []
    for speakerid in ids:
        text = speakers.ix[speakerid,'whylisten'] + speakers.ix[speakerid,'whotheyare']
        gender.append(genderdet(text))
    if len(gender) == 1:
        return gender[0]
    return gender

In [33]:
gender = talks_train['speaker_ids'].apply(speakerinfo)

In [34]:
gender.value_counts()

/Users/ysfeng/anaconda/lib/python2.7/site-packages/pandas/core/algorithms.py:245: RuntimeWarning: tp_compare didn't return -1 or -2 for exception
  keys, counts = htable.value_count_object(values, mask)


1            1030
0             405
-1             72
[1, 1]         13
[0, 1]          4
[0, 0]          3
[-1, 1]         2
[1, 0]          2
[1, 0, 1]       1
[-1, -1]        1
[0, 0, 0]       1
[-1, 0]         1
dtype: int64

In [35]:
gender

0            1
1            0
2            1
3            1
4            0
5            1
6            1
7            1
8            1
9            1
10           1
11           1
12           1
13           1
14           1
15          -1
16           1
17           0
18           0
19          -1
20           1
21           1
22           1
23      [1, 1]
24           1
25           1
26           1
27           1
28           1
29           0
         ...  
1505        -1
1506         1
1507         1
1508         1
1509         0
1510         1
1511         1
1512         1
1513         1
1514         0
1515         1
1516         1
1517         1
1518         0
1519         1
1520         0
1521         1
1522         1
1523         1
1524         1
1525    [0, 0]
1526         1
1527         1
1528         1
1529         0
1530         1
1531         1
1532         0
1533         1
1534         0
Name: speaker_ids, dtype: object

#### Resort to gender.api.com to check the remaining speakers using their first name 

In [36]:
# row index of speaker gender-unidentified talks
unidentified = list(gender[gender == -1].index)

In [37]:
unidentified[0]

15

In [38]:
baseurl = 'https://gender-api.com/get?name='
key='oxxyvSjsHaSKyuLUFF'

In [39]:
def checkfirstname(ind):
    # ind is the row indices of gender(talks_train)
    # since in this case we know these talks have only one speaker, we just assign it to speakerid as a variable
    speakerid = talks_train.ix[ind,'speaker_ids'][0]
    name = speakers.ix[speakerid,'firstname']
    url = baseurl+name
    response = requests.get(url,params = dict(key=key))
    return response.json()['gender']

In [40]:
checkfirstname(15)

u'male'

In [41]:
sex = [checkfirstname(x) for x in unidentified]

In [42]:
sex_bin = []
for det in sex:
    if det == 'male':
        sex_bin.append(1)
    elif det == 'female':
        sex_bin.append(0)
    else:
        sex_bin.append(-1)

In [43]:
sex_bin = pd.Series(sex_bin,index = unidentified)

In [44]:
sex_bin

15      1
19      1
50      1
98      1
102     1
109     1
138     1
149     1
153     1
185     1
191     1
226     1
258     1
260     1
270     1
279     0
295     1
296    -1
367     1
404    -1
432     1
438     1
511     1
528     1
558     1
613     0
626     0
627     1
649     1
654     1
       ..
991     1
992     0
1011    1
1025   -1
1027    1
1045    1
1058    1
1067    1
1080    1
1085    0
1093    1
1110    1
1213   -1
1223    1
1242    1
1262    1
1281    1
1295   -1
1360   -1
1369    1
1374    1
1375    1
1376    0
1397    1
1418    0
1422    1
1426    1
1481    1
1498    0
1505   -1
dtype: int64

In [45]:
gender[unidentified]=sex_bin

In [46]:
gender.value_counts()

1            1084
0             415
[1, 1]         13
-1              8
[0, 1]          4
[0, 0]          3
[-1, 1]         2
[1, 0]          2
[1, 0, 1]       1
[-1, -1]        1
[0, 0, 0]       1
[-1, 0]         1
dtype: int64

In [47]:
gender[gender == -1].index

Int64Index([296, 404, 817, 1025, 1213, 1295, 1360, 1505], dtype='int64')

In [48]:
talks_train.ix[1300,'name']

u'Stewart Brand + Mark Z. Jacobson: Debate: Does the world need nuclear energy?'

In [49]:
gender[296]=1

In [50]:
gender[404]=0;gender[817]=1;gender[1025]=1;

In [51]:
gender[1213]=0;gender[1295]=1;gender[1360]=0;

In [52]:
gender[1505]=1

In [53]:
gender.value_counts()

1            1089
0             418
[1, 1]         13
[0, 1]          4
[0, 0]          3
[-1, 1]         2
[1, 0]          2
[1, 0, 1]       1
[-1, -1]        1
[0, 0, 0]       1
[-1, 0]         1
dtype: int64

In [54]:
for ind, sex in gender.iteritems():
    if type(sex)==list:
        if len(sex)==2 and sum(sex)==2:
            gender[ind]=1
        elif len(sex)==3 and sum(sex)==0:
            gender[ind]=0
        elif len(sex)==3 and sum(sex)==2:
            gender[ind]=1
        elif len(sex)==2 and sex[0]==0 and sex[1]==0:
            gender[ind]=0
        else:
            print ind,sex

166 [0, 1]
196 [-1, 0]
262 [0, 1]
441 [1, 0]
657 [0, 1]
820 [-1, 1]
897 [-1, -1]
976 [0, 1]
1260 [1, 0]
1300 [-1, 1]


In [55]:
gender[196]=1;gender[820]=1;gender[897]=1;gender[1300]=1

In [56]:
for ind, sex in gender.iteritems():
    if type(sex)==list:
        print ind,sex

166 [0, 1]
262 [0, 1]
441 [1, 0]
657 [0, 1]
976 [0, 1]
1260 [1, 0]


In [57]:
gender.ix[[166,262,441,657,976,1260]]=1

In [58]:
gender.value_counts()

1    1113
0     422
dtype: int64

###gender determination summary: 

1. Most of them are determined by looking for he/she etc in the whytohear, whotheyare columns
2. The unknown from the 1st step are determined by first name check on gendercheck.api.com
3. The remaining unknowns are determined manually by watching the video
4. In the case of multiple speakers: (1) Majority gender (2)If even, choose 'male' or 1
5. Notice there are many cases where there are multiple speakers but TED recorded as one speaker.

In [59]:
gender.shape

(1535,)

In [60]:
talks_train['speaker_gender']=gender

In [61]:
talks_train.head()

,Beautiful,Confusing,Courageous,Fascinating,Funny,Informative,Ingenious,Inspiring,Jaw-dropping,Longwinded,...,native_language_code,published_at,rating_count,recorded_at,released_at,speaker_ids,themes,updated_at,viewed_count,speaker_gender
0,50,51,126,100,460,383,50,356,85,95,...,en,2006-06-27 00:11:00,2485,2006-02-25,2013-12-02 21:24:10,[2],"[{u'theme': {u'id': 7, u'name': u'Presentation...",2014-11-05 16:39:29,2842062,1
1,45,NaN,128,139,15,236,269,432,61,14,...,en,2006-08-15 00:11:00,1511,2006-02-24,2013-12-12 20:29:35,[3],"[{u'theme': {u'id': 15, u'name': u'A Greener F...",2015-04-11 07:03:32,956392,0
2,21,5,27,201,14,322,242,326,23,26,...,en,2015-01-12 16:15:47,1398,2014-10-09,2015-01-12 16:15:47,[2046],"[{u'theme': {u'id': 28, u'name': u'Not Busines...",2015-05-08 18:50:06,1117078,1
3,37,22,123,38,9,187,27,100,29,18,...,en,2006-10-18 00:11:00,772,2005-07-12,2014-04-11 04:08:46,[4],"[{u'theme': {u'id': 20, u'name': u'Rethinking ...",2014-08-05 20:18:08,650873,1
4,73,5,12,6,229,8,41,19,5,6,...,en,2007-04-06 00:30:00,531,2006-02-26,2014-09-23 22:44:38,[105],"[{u'theme': {u'id': 9, u'name': u'Spectacular ...",2014-09-23 22:44:38,441840,0


###Here also generate a feature to record multi-speaker talks 

In [62]:
onespeaker = talks_train.speaker_ids.apply(lambda x:len(x)==1)

In [63]:
sum(~onespeaker)

28

In [64]:
talks_train['onespeaker']=onespeaker

In [65]:
talks_train.to_json('train2.json')

In [66]:
talks_train.columns

Index([           u'Beautiful',            u'Confusing',
                 u'Courageous',          u'Fascinating',
                      u'Funny',          u'Informative',
                  u'Ingenious',            u'Inspiring',
               u'Jaw-dropping',           u'Longwinded',
                         u'OK',            u'Obnoxious',
                 u'Persuasive',         u'Unconvincing',
            u'commented_count',          u'description',
              u'emailed_count',             u'event_id',
                         u'id',             u'keywords',
                  u'languages',                 u'name',
       u'native_language_code',         u'published_at',
               u'rating_count',          u'recorded_at',
                u'released_at',          u'speaker_ids',
                     u'themes',           u'updated_at',
               u'viewed_count',       u'speaker_gender',
                 u'onespeaker'],
      dtype='object')

##2. Process talk text information: title + keywords + description

This is the first of all text resources that will be tapped on, the second would be the captions (e.g., first and last paragraphs); the third will be speakers' background info

In [9]:
talks_train = pd.read_json('train2.json')

In [10]:
labels = pd.read_csv('response5.csv',header =None )

In [11]:
labels.shape

(1535, 2)

In [12]:
labels.columns = ['id','label']

In [13]:
talks = pd.merge(talks_train[['description',
                             'id', 'keywords', 'name']],labels,on = 'id')

In [14]:
talks.sort(inplace = True, columns = 'id')

In [15]:
talks.reset_index(inplace = True, drop = True)

In [16]:
def gettitle(name):
    tmp = name.split(':')
    if len(tmp)==1:
        return tmp[0]
    else:
        return tmp[1]

In [17]:
talks['title'] = talks.name.apply(gettitle)

In [18]:
talks.drop('name',axis = 1,inplace = True)

In [19]:
talks.head()

,description,id,keywords,label,title
0,With the same humor and humanity he exuded in ...,1,"Presentation Innovation,A Greener Future?,Bold...",4,Averting the climate crisis
1,Fumes from indoor cooking fires kill more than...,2,"A Greener Future?,Tales of Invention,Rethinkin...",1,Simple designs to save a life
2,Ashraf Ghani's passionate and powerful 10-minu...,3,"Rethinking Poverty,Africa: The Next Chapter,Wa...",0,How to rebuild a broken state
3,"In this passionate talk, legendary spacecraft ...",4,"What's Next in Tech,Tales of Invention,To Bold...",0,The real future of space exploration
4,Genomics pioneer Craig Venter takes a break fr...,6,"Animals That Amaze,Evolution's Genius,Inspired...",2,Sampling the ocean's DNA


In [20]:
text = []
for ind, row in talks.iterrows():
    text.append(' '.join([row['description'],row['keywords'],row['title']]))

In [33]:
talks['text'] = text

In [34]:
talks.text[0]

u'With the same humor and humanity he exuded in <em>An Inconvenient Truth,</em> Al Gore spells out 15 ways that individuals can address climate change immediately, from buying a hybrid to inventing a new, hotter "brand name" for global warming. Presentation Innovation,A Greener Future?,Bold Predictions, Stern Warnings,Design That Matters,Not Business as Usual,Technology, History and Destiny,TED Conference,alternative energy,cars,climate change,culture,environment,global issues,politics,science,sustainability,technology  Averting the climate crisis'

###2.1 Remove tags  

In [35]:
talks['text'] = talks['text'].apply(lambda x: re.sub("<.*?>", "", x))    

In [36]:
talks.text[0]

u'With the same humor and humanity he exuded in An Inconvenient Truth, Al Gore spells out 15 ways that individuals can address climate change immediately, from buying a hybrid to inventing a new, hotter "brand name" for global warming. Presentation Innovation,A Greener Future?,Bold Predictions, Stern Warnings,Design That Matters,Not Business as Usual,Technology, History and Destiny,TED Conference,alternative energy,cars,climate change,culture,environment,global issues,politics,science,sustainability,technology  Averting the climate crisis'

###2.2 Tokenize, remove punctuation, lowercase and stop-words removed

In [25]:
from string import punctuation

In [26]:
pun_list = list(punctuation)

In [27]:
stemmer = SnowballStemmer('english',ignore_stopwords = True)

def tlss(text):
    # tokenize words
    words = word_tokenize(text)
    filtered_words = []
    for word in words:
        # remove punctuation and numbers
        if re.search('[a-zA-Z]', word):
            # lower case
            filtered_words.append(word.lower())
    stemmed = [stemmer.stem(x) for x in filtered_words]
    stemmed = set(stemmed).difference(set(stopwords.words('english')))
    return list(stemmed)

In [28]:
talks['text'] = talks['text'].apply(tlss)

### Consider using countvectorizer directly 

In [37]:
vect = CountVectorizer(stop_words = 'english',ngram_range = (1,3),min_df = 0.2, binary = True,max_features = 100)

In [38]:
textgroupby = talks.groupby('label')['text']

In [39]:
transformed = []
for ind, textgb in textgroupby:
    print ind, (textgb.as_matrix()).shape
    textarray = vect.fit_transform(textgb.as_matrix())
    vocab = vect.vocabulary_
    transformed.append(dict(text = textarray, index = ind, vocab = vocab))

0 (474,)
1 (418,)
2 (353,)
3 (179,)
4 (111,)


In [135]:
transformed[0]

{'index': 0, 'text': <474x21 sparse matrix of type '<type 'numpy.int64'>'
 	with 2529 stored elements in Compressed Sparse Row format>, 'vocab': {u'bold': 0,
  u'bold predictions': 1,
  u'bold predictions stern': 2,
  u'business': 3,
  u'conference': 4,
  u'culture': 5,
  u'global': 6,
  u'global issues': 7,
  u'issues': 8,
  u'predictions': 9,
  u'predictions stern': 10,
  u'predictions stern warnings': 11,
  u'science': 12,
  u'stern': 13,
  u'stern warnings': 14,
  u'talk': 15,
  u'technology': 16,
  u'ted': 17,
  u'ted conference': 18,
  u'warnings': 19,
  u'world': 20}}

In [145]:
testmx = transformed[4]['text']

In [148]:
testmx2 = testmx.toarray()

In [149]:
testmx2.shape

(111, 14)

In [150]:
testmx2.sum(axis = 0 )

array([29, 28, 60, 33, 30, 39, 39, 40, 30, 34, 63, 59, 23, 23])

In [151]:
text_words = [x['vocab'] for x in transformed]

In [182]:
themes = [u'Live Music',
 u'Celebrating TEDWomen',
 u'Whipsmart Comedy',
 u'Listening to Teachers',
 u'Spectacular Performance',
 u'Ocean Wonders',
 u'Bold Predictions, Stern Warnings',
 u'What Makes Us Happy?',
 u'Media With Meaning',
 u'Technology, History and Destiny',
 u'How the Mind Works',
 u'Talks from TED Fellows',
 u'Medicine Without Borders',
 u'Hidden Gems',
 u'War and Peace',
 u'The Rise of Collaboration',
 u'TED Prize Winners',
 u"Evolution's Genius",
 u'Unconventional Explanations',
 u"What's Next in Tech",
 u'Presentation Innovation',
 u'Words About Words',
 u'Numbers at Play',
 u'Women Reshaping the World',
 u'Art Unusual',
 u'Not Business as Usual',
 u'To Boldly Go ...',
 u'Architectural Inspiration',
 u'Africa: The Next Chapter',
 u'TED in 3 Minutes',
 u'Tales of Invention',
 u'Inspired by Nature',
 u'Is There a God?',
 u'Design That Matters',
 u'Food Matters',
 u'How We Learn',
 u'Master Storytellers',
 u'Animals That Amaze',
 u'The Power of Cities',
 u'A Greener Future?',
 u'The Creative Spark',
 u'Peering into Space',
 u'Best of the Web',
 u'Rethinking Poverty',
 u'The Charter for Compassion',
 u'Might You Live a Great Deal Longer?',
 u'Mission Blue Voyage']

In [183]:
themes = ' '.join(themes)

In [184]:
themes = themes.lower()

####Obviously, "ted conference" are not predictive, put them also into themes 

In [185]:
# add 'ted conference' to the themes
themes = ' '.join([themes,'ted conference'])

In [186]:
# define a function to search through themes, the keys in each text_words dict
def words_not_themes(textwords):
    filtered_words = []
    for key in textwords:
        match = re.search(key,themes)
        if not match:
            filtered_words.append(key)
    return filtered_words

In [187]:
filtered_text_words = [words_not_themes(x) for x in text_words]

In [188]:
filtered_text_words

[[u'global issues',
  u'science',
  u'global',
  u'bold predictions stern',
  u'culture',
  u'issues',
  u'predictions stern',
  u'predictions stern warnings'],
 [u'culture', u'global issues', u'global', u'issues'],
 [u'tales invention', u'science', u'new'],
 [u'global', u'culture', u'global issues', u'issues'],
 [u'humor', u'entertainment', u'culture']]

by comparing the filtered_text_words list with themes, the final list of text words is

In [201]:
final_text_words = ['global issues', 'science', 'culture', 'new','humor','entertainment']

### With this small vocabulary count their frequencies in each text 

In [190]:
talks.text.shape

(1535,)

In [197]:
talks[['id','text']].to_json('talks_other_text.json')

In [202]:
vect1 = CountVectorizer(vocabulary = final_text_words, stop_words = 'english', ngram_range = (1,2))

In [203]:
other_text_feature = vect1.fit_transform(talks['text'].as_matrix())

In [204]:
other_text_feature.shape

(1535, 6)

In [205]:
other_text_feature.toarray()[0,:]

array([1, 1, 1, 1, 1, 0])

In [206]:
talks['text'][0]

u'With the same humor and humanity he exuded in An Inconvenient Truth, Al Gore spells out 15 ways that individuals can address climate change immediately, from buying a hybrid to inventing a new, hotter "brand name" for global warming. Presentation Innovation,A Greener Future?,Bold Predictions, Stern Warnings,Design That Matters,Not Business as Usual,Technology, History and Destiny,TED Conference,alternative energy,cars,climate change,culture,environment,global issues,politics,science,sustainability,technology  Averting the climate crisis'

In [207]:
vect1.vocabulary_

{u'culture': 2,
 u'entertainment': 5,
 u'global issues': 0,
 u'humor': 4,
 u'new': 3,
 u'science': 1}